In [1]:
import os
import json
import re
import tempfile
from pdf2image import convert_from_path
import fitz
from PIL import Image
import io
import pandas as pd
import time
from langchain_upstage import UpstageLayoutAnalysisLoader
from rainbow_html_transformer import HTMLToTextWithMarkdownTables
from tqdm import tqdm

In [2]:
def get_pdf_pages(pdf_path):
    doc = fitz.open(pdf_path)
    return len(doc)

In [3]:
def extract_text_with_ocr(pdf_path, max_pages=3):
    extracted_text = []
    try:
        # PDF를 이미지로 변환
        images = convert_from_path(pdf_path, first_page=1, last_page=max_pages)
        
        print(f"Processing {len(images)} pages...")
        for i, img in enumerate(tqdm(images, desc="Processing pages", unit="page")):
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as temp_file:
                img.save(temp_file, format="PNG")
                temp_file_path = temp_file.name
            
            try:
                loader = UpstageLayoutAnalysisLoader(
                    file_path=temp_file_path,
                    use_ocr=True
                )
                documents = loader.load()
                
                html_transformer = HTMLToTextWithMarkdownTables()
                for doc in documents:
                    transformed_doc = html_transformer.transform_documents([doc])[0]
                    # 페이지 변경 정보 추가
                    page_content = transformed_doc.page_content
                    page_content += f"\n\n--- End of Page {i+1} ---\n\n"
                    extracted_text.append(page_content)
            finally:
                os.unlink(temp_file_path)
    
    except Exception as e:
        print(f"Error processing {pdf_path} with OCR: {e}")
    
    return ''.join(extracted_text)

In [4]:
# 청킹을 위한 기준 정규표현식 패턴
def create_section_patterns(contract_name):
    return {
        "약관 가이드북": r"^\s*약관 가이드북\s*$",
        "약관 요약서": r"^\s*약관 요약서\s*$",
        "주요보험용어 해설": r"^\s*주요보험용어 해설\s*$",
        "가입부터 지급까지 쉽게 찾기": r"^\s*가입부터 지급까지 쉽게 찾기\s*$",
        "주계약": fr"^\s*{re.escape(contract_name)}\s*$",
        "부표": r"^\[부표\d+\]",
        "특약": r".+특약\(.+\)",
        "별첨": r"^\[별첨\d+\]"
    }

In [5]:
# 텍스트 파일에서 내용을 불러오는 함수
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [6]:
# 문서 주제를 페이지 단위로 찾는 함수
def find_document_title(text):
    # 텍스트를 페이지 단위로 분할
    pages = text.split('--- End of Page')
    
    # 각 페이지에서 "목 차"와 "제1관 목적 및 용어의 정의" 사이의 텍스트를 찾기
    for page in pages:
        title_pattern = r"목\s*차(.*?)제1관 목적 및 용어의 정의"
        match = re.search(title_pattern, page, re.DOTALL)
        if match:
            potential_titles = match.group(1).strip().splitlines()
            for line in reversed(potential_titles):
                if line.strip():  # 비어 있지 않은 마지막 라인을 문서 주제로 설정
                    return line.strip()
    
    return "Unknown Document Title"


In [7]:
# 주계약 명칭을 찾는 함수
def find_contract_name(text):
    # 목차에서 "가입부터 지급까지 쉽게 찾기" 아래의 텍스트를 찾기
    match = re.search(r"가입부터\s*지급까지\s*쉽게\s*찾기\s*\d+\s*(.*?)\s*제1관", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # 목차에서 직접 "제1관" 라인 바로 전의 텍스트를 찾기
    match = re.search(r"(.*?)\s*제1관", text, re.DOTALL)
    if match:
        potential_titles = match.group(1).strip().splitlines()
        if potential_titles:
            return potential_titles[-1].strip()
    
    return "Unknown Contract Name"

In [8]:
# 텍스트를 청킹하는 함수
def chunk_document(text, section_patterns, document_title, chunk_size=1000):
    chunks = []
    page_number = 1  # 페이지 번호 초기화
    current_section = {
        "문서주제": document_title, 
        "섹션 대분류": None, 
        "섹션 중분류": None, 
        "섹션 소분류": None, 
        "페이지": page_number, 
        "청킹내용": ""
    }
    
    pages = text.split('--- End of Page')
    in_overview_section = True  # 문서개요 구분

    for page in pages:
        lines = page.split('\n')
        for line in lines:
            # 문서개요 부분 청킹
            if in_overview_section:
                if any(re.match(pattern, line.strip()) for pattern in section_patterns.values()):
                    in_overview_section = False  # 문서개요가 끝나고 다음 섹션으로 이동
                    break
                else:
                    current_section["청킹내용"] += line.strip() + " "
                    if len(current_section["청킹내용"]) >= chunk_size:
                        chunk = current_section.copy()
                        chunk["섹션 대분류"] = "문서개요"
                        chunk["섹션 중분류"] = "문서개요"
                        chunk["섹션 소분류"] = "문서개요"
                        chunks.append(chunk)
                        current_section["청킹내용"] = ""
            else:
                # 패턴에 따라 섹션 구분
                for section_name, pattern in section_patterns.items():
                    if re.match(pattern, line.strip()):
                        # 기존 청킹된 내용이 있으면 저장
                        if current_section["청킹내용"].strip():
                            chunks.append(current_section.copy())

                        # 새로운 섹션 시작
                        current_section["섹션 대분류"] = section_name
                        current_section["섹션 중분류"] = line.strip()
                        current_section["섹션 소분류"] = None
                        
                        # 페이지 정보 업데이트
                        current_section["페이지"] = page_number
                        current_section["청킹내용"] = ""
                        break
                else:
                    # 현재 섹션에 내용을 추가
                    current_section["청킹내용"] += line.strip() + " "
        
        # 페이지가 끝날 때마다 페이지 번호 증가
        page_number += 1
    
    # 마지막 청킹된 내용 저장
    if current_section["청킹내용"].strip():
        chunks.append(current_section)
    
    return chunks


In [9]:
# 청킹된 내용을 엑셀로 저장하는 함수
def save_chunks_to_excel(chunks, output_path):
    df = pd.DataFrame(chunks)
    df.to_excel(output_path, index=False)

In [10]:
def main():
    folder_path = "./raw_docs"
    output_folder_path = "./processed_txt"
    # pdf_file_name = "판매약관_ThePride신한참좋은치아보험PlusⅡ(무배당, 갱신형)_20240401_P252.pdf"
    pdf_file_name = "SHL0165_The안심VIP저축보험Ⅱ(무배당)_P116.pdf"

    print(f"Processing file: {pdf_file_name}")
    pdf_path = os.path.join(folder_path, pdf_file_name)

    max_pages = get_pdf_pages(pdf_path)
    ocr_text = extract_text_with_ocr(pdf_path, max_pages)

    # 텍스트 파일로 저장
    output_file_name = os.path.splitext(pdf_file_name)[0] + ".txt"
    text_output_path = os.path.join(output_folder_path, output_file_name)

    with open(text_output_path, 'w', encoding='utf-8') as f:
        f.write(ocr_text)

    print(f"OCR text saved to: {text_output_path}")

    # 저장된 텍스트 출력 (선택사항)
    print("Extracted text:")
    print(ocr_text[:500] + "...")  # 처음 500자만 출력

    # 저장된 텍스트 파일 로드
    text = load_text_file(text_output_path)
    
    # 문서 주제 찾기
    document_title = find_document_title(text)
    
    # 주계약 명칭 찾기
    contract_name = find_contract_name(text)
    
    # 청킹을 위한 패턴 생성
    section_patterns = create_section_patterns(contract_name)
    
    # 문서 청킹
    chunks = chunk_document(text, section_patterns, document_title)
    
    # 엑셀로 저장
    excel_output_path = os.path.join(output_folder_path, 'chunked_insurance_document.xlsx')
    save_chunks_to_excel(chunks, excel_output_path)
    print(f"청킹된 문서가 {excel_output_path}에 저장되었습니다.")

# 프로그램 실행
main()

Processing file: SHL0165_The안심VIP저축보험Ⅱ(무배당)_P116.pdf
Processing 116 pages...


Processing pages: 100%|██████████| 116/116 [05:59<00:00,  3.10s/page]


OCR text saved to: ./processed_txt/SHL0165_The안심VIP저축보험Ⅱ(무배당)_P116.txt
Extracted text:
약관
The안심VIP저축보험�
(무배당)

--- End of Page 1 ---

고객에게 드리는 감사의 인사말씀
먼저, 변함없는 신뢰와 성원을 보내주신 고객여러분께
진심으로 깊은 감사의 말씀을 드립니다.
2021년 7월 새롭게 태어난 신한라이프는
대한민국 생명보험업계를 선도하는 一流 리딩 컴퍼니를 지향합니다.
신한라이프는 'NewLife, Life에 새로운 가치를 더하다' 라는
비전을 가지고 있습니다.
생명보험업계 최고 수준의 재무건전성을 유지하고,
고객을 이롭게 하는 새로운 보험상품을 개발하고
모바일 기반의 디지털 전환과 헬스케어 서비스 확대를 통하여
'고객님의 삶에 새로운 가치를 더하는' 신한라이프가 되겠습니다.
나아가 '금융으로 세상을 이롭게 한다'는 회사의 미션을 실천하고
나눔과 상생으로 소외된 이웃에게 힘이 되며,
ESG환경사회자배구조Ervironment Scote,Govermance) 중심 경영을 통해
우리사회의 발전에 지속적으로 기여하는 회사가 되겠습니다.
앞으로도 ...
청킹된 문서가 ./processed_txt/chunked_insurance_document.xlsx에 저장되었습니다.
